In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Städte in Deutschland
cities = [
    "Berlin", "Hamburg", "München", "Köln", "Frankfurt", 
    "Stuttgart", "Düsseldorf", "Dortmund", "Essen", "Leipzig"
]

# Zeitraum: Letzte 10 Jahre (täglich)
start_date = datetime(2015, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Windrichtungen
wind_directions = ["N", "NE", "E", "SE", "S", "SW", "W", "NW"]

# Funktion zur Generierung synthetischer Wetterdaten
def generate_weather_data():
    data = []

    for city in cities:
        for date in date_range:
            month = date.month

            # Jahreszeitabhängige Temperatur
            if month in [12, 1, 2]:      # Winter
                temp = np.random.normal(1, 5)
            elif month in [3, 4, 5]:     # Frühling
                temp = np.random.normal(10, 5)
            elif month in [6, 7, 8]:     # Sommer
                temp = np.random.normal(20, 5)
            else:                        # Herbst
                temp = np.random.normal(10, 5)

            # Abgeleitete Daten
            feels_like = temp + np.random.normal(0, 2)
            precipitation = max(0, np.random.exponential(1.5) if random.random() < 0.3 else 0)
            rain = precipitation > 0 and temp > 1
            snow = precipitation > 0 and temp <= 1
            wind_speed = np.random.normal(15, 5)
            wind_dir = random.choice(wind_directions)

            data.append({
                "Stadt": city,
                "Datum": date.strftime("%Y-%m-%d"),
                "Temperatur_C": round(temp, 1),
                "Gefuehlte_Temperatur_C": round(feels_like, 1),
                "Niederschlag_mm": round(precipitation, 1),
                "Regen": rain,
                "Schnee": snow,
                "Wind_kmh": round(max(0, wind_speed), 1),
                "Windrichtung": wind_dir
            })

    return pd.DataFrame(data)

# Daten generieren
df = generate_weather_data()

# Vorschau
print(df.head())

# Als CSV speichern
df.to_csv("synthetisches_wetter_deutschland.csv", index=False)
print("Datei 'synthetisches_wetter_deutschland.csv' wurde erfolgreich erstellt.")

    Stadt       Datum  Temperatur_C  Gefuehlte_Temperatur_C  Niederschlag_mm  \
0  Berlin  2015-01-01          -1.0                    -2.0              0.0   
1  Berlin  2015-01-02           3.1                     3.1              0.0   
2  Berlin  2015-01-03           4.4                     6.1              0.0   
3  Berlin  2015-01-04           2.6                     3.4              2.0   
4  Berlin  2015-01-05           9.7                     8.8              0.2   

   Regen  Schnee  Wind_kmh Windrichtung  
0  False   False      20.5           NE  
1  False   False      16.3           NW  
2  False   False      23.8            E  
3   True   False      18.1           NW  
4   True   False      12.0           SW  
Datei 'synthetisches_wetter_deutschland.csv' wurde erfolgreich erstellt.


In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import datetime

# 1. CSV einlesen
df = pd.read_csv("synthetisches_wetter_deutschland.csv")

# 2. Feature Engineering
df['Datum'] = pd.to_datetime(df['Datum'])
df['Jahr'] = df['Datum'].dt.year
df['Monat'] = df['Datum'].dt.month
df['Tag'] = df['Datum'].dt.day

# Stadt als Zahl encodieren
le_stadt = LabelEncoder()
df['Stadt_encoded'] = le_stadt.fit_transform(df['Stadt'])

# Zielvariablen
targets = ['Temperatur_C', 'Gefuehlte_Temperatur_C', 'Niederschlag_mm', 'Wind_kmh']

# Eingabefeatures
X = df[['Stadt_encoded', 'Jahr', 'Monat', 'Tag']]
y = df[targets]

# 3. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Modell trainieren
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5. Modellbewertung
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE für Wetterprognose: {mae:.2f} 🌡️")

# 6. Wetter vorhersagen + Kommentar generieren
def vorhersage(stadt_name, datum_str):
    datum = pd.to_datetime(datum_str)
    eingabe = pd.DataFrame({
        "Stadt_encoded": [le_stadt.transform([stadt_name])[0]],
        "Jahr": [datum.year],
        "Monat": [datum.month],
        "Tag": [datum.day]
    })

    prognose = model.predict(eingabe)[0]
    temp, feels_like, rain, wind = prognose

    # Lustiger Kommentar basierend auf Wetter
    kommentar = "☀️ Schönes Wetter!"
    if temp < 0:
        kommentar = "❄️ Frostiger Tag! Zeit für Glühwein?"
    elif rain > 5:
        kommentar = "☔ Bring deinen Regenschirm mit!"
    elif wind > 30:
        kommentar = "💨 Achtung, stürmisch heute!"

    # Ausgabe
    print(f"\n📍 Stadt: {stadt_name}, 📅 Datum: {datum_str}")
    print(f"🌡️ Temperatur: {temp:.1f}°C, Gefühlte: {feels_like:.1f}°C")
    print(f"🌧️ Niederschlag: {rain:.1f} mm, 💨 Wind: {wind:.1f} km/h")
    print(f"🗯️ Kommentar: {kommentar}")

# 7. Beispiel verwenden
vorhersage("Berlin", "2025-03-25")


MAE für Wetterprognose: 3.54 🌡️

📍 Stadt: Berlin, 📅 Datum: 2025-03-25
🌡️ Temperatur: 9.5°C, Gefühlte: 9.6°C
🌧️ Niederschlag: 0.1 mm, 💨 Wind: 15.3 km/h
🗯️ Kommentar: ☀️ Schönes Wetter!


In [6]:
from sklearn.metrics import mean_squared_error, r2_score

# 5.1 Erweiterte Modellbewertung
def bewertung(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print("\n📈 Modellbewertung:")
    print(f"🔹 MAE:  {mae:.2f}")
    print(f"🔹 RMSE: {rmse:.2f}")
    print(f"🔹 R²:   {r2:.3f}")

# Aufrufen für gesamtes Modell
bewertung(y_test, y_pred)



📈 Modellbewertung:
🔹 MAE:  3.54
🔹 RMSE: 4.89
🔹 R²:   0.184


In [9]:
for i, ziel in enumerate(targets):
    print(f"\n🔍 Bewertung für: {ziel}")
    bewertung(y_test.iloc[:, i], y_pred[:, i])



🔍 Bewertung für: Temperatur_C

📈 Modellbewertung:
🔹 MAE:  4.43
🔹 RMSE: 5.53
🔹 R²:   0.568

🔍 Bewertung für: Gefuehlte_Temperatur_C

📈 Modellbewertung:
🔹 MAE:  4.73
🔹 RMSE: 5.93
🔹 R²:   0.528

🔍 Bewertung für: Niederschlag_mm

📈 Modellbewertung:
🔹 MAE:  0.71
🔹 RMSE: 1.14
🔹 R²:   -0.183

🔍 Bewertung für: Wind_kmh

📈 Modellbewertung:
🔹 MAE:  4.29
🔹 RMSE: 5.36
🔹 R²:   -0.179


In [12]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)


In [14]:
# Datum als datetime interpretieren
df['Datum'] = pd.to_datetime(df['Datum'])

# Neue Features aus Datum
df['Monat'] = df['Datum'].dt.month
df['Wochentag'] = df['Datum'].dt.weekday
df['Jahreszeit'] = df['Monat'] % 12 // 3 + 1  # 1: Winter, 2: Frühling, 3: Sommer, 4: Herbst


In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Merkmale & Ziel
features = ['Gefuehlte_Temperatur_C', 'Niederschlag_mm', 'Wind_kmh', 
            'Regen', 'Schnee', 'Monat', 'Wochentag', 'Jahreszeit']
target = 'Temperatur_C'

X = df[features]
y = df[target]

# Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Modell
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Vorhersage & Bewertung
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("📈 Neue Modellbewertung (Random Forest):")
print(f"🔹 MAE:  {mae:.2f}")
print(f"🔹 RMSE: {rmse:.2f}")
print(f"🔹 R²:   {r2:.3f}")


📈 Neue Modellbewertung (Random Forest):
🔹 MAE:  1.54
🔹 RMSE: 1.95
🔹 R²:   0.947


In [18]:
import random

def wetter_kommentar(temp):
    if temp < 0:
        return "🥶 Brrr, zieh dich warm an!"
    elif temp < 10:
        return "🧥 Kalt! Vielleicht ein Tee?"
    elif temp < 20:
        return "🌤 Frisch, aber okay!"
    elif temp < 30:
        return "☀️ Perfektes Wetter für draußen!"
    else:
        return "🔥 Hitzefrei gefällig?"

def vorhersage(stadt, datum_str):
    datum = pd.to_datetime(datum_str)
    stadt_df = df[df['Stadt'] == stadt]

    # Mittelwerte der anderen Features aus der Stadt für das gegebene Datum
    input_data = {
        'Gefuehlte_Temperatur_C': stadt_df['Gefuehlte_Temperatur_C'].mean(),
        'Niederschlag_mm': stadt_df['Niederschlag_mm'].mean(),
        'Wind_kmh': stadt_df['Wind_kmh'].mean(),
        'Regen': random.choice([0, 1]),
        'Schnee': random.choice([0, 1]),
        'Monat': datum.month,
        'Wochentag': datum.weekday(),
        'Jahreszeit': datum.month % 12 // 3 + 1
    }

    # Umwandeln in DataFrame für Modell
    input_df = pd.DataFrame([input_data])
    prediction = model.predict(input_df)[0]

    kommentar = wetter_kommentar(prediction)
    print(f"📍 Stadt: {stadt}, 📅 Datum: {datum.date()}")
    print(f"🌡️ Vorhergesagte Temperatur: {prediction:.2f} °C")
    print(f"💬 Kommentar: {kommentar}")

# Beispiel:
vorhersage("Berlin", "2025-04-15")


📍 Stadt: Berlin, 📅 Datum: 2025-04-15
🌡️ Vorhergesagte Temperatur: 9.89 °C
💬 Kommentar: 🧥 Kalt! Vielleicht ein Tee?


In [20]:
vorhersage("Leipzig", "2025-07-15")

📍 Stadt: Leipzig, 📅 Datum: 2025-07-15
🌡️ Vorhergesagte Temperatur: 11.19 °C
💬 Kommentar: 🌤 Frisch, aber okay!
